In [1]:
#Reading data
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
scaling=MinMaxScaler()
df= pd.read_json("product_dataset_final.json")
ff = open('new_user_rating_demo.json')
ratingArray = ff.read()
ff.close()

In [2]:
df.head()

,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,rating,ratingcount
0,[Keep your air humidifier operating at peak ef...,"Holmes &quot;C&quot; Humidifier Filter, HWF65P...",Holmes,Improves efficiency of the humidifier Extende...,Amazon Home,20080101,5.98,B00005OU6T,[https://images-na.ssl-images-amazon.com/image...,3.820000,250
1,"[Holmes ""A"" Humidifier Filter, HWF62]","Holmes &quot;A&quot; Humidifier Filter, HWF62",Holmes,"Nullify Imported Fits Holmes, Sunbeam, and Bi...",Amazon Home,20080101,11.89,B00006IV17,[https://images-na.ssl-images-amazon.com/image...,4.326797,765
2,"[The Holmes cool mist filter transforms dry, d...","Holmes &quot;B&quot; Humidifier Filter, HWF64",Holmes,"Fits Holmes, Sunbeam, and Bionaire Humidifier...",Amazon Home,20080101,11.76,B00007E7RQ,[https://images-na.ssl-images-amazon.com/image...,4.126531,245
3,"[, <br />Want a fresh start with your stove bu...",Range Kleen 8234 Gas Range Knobs &ndash; 4 Whi...,Range Kleen,SAFE GAS RANGE REPLACEMENT KNOBS: Gas range k...,Appliances,20030514,12.66,B00009V3UF,[https://images-na.ssl-images-amazon.com/image...,2.634615,52
4,[Indoor dryer venting supurr-flex UL listed 21...,"Deflecto Dryer Lint Trap Kit, Supurr-Flex Flex...",Deflecto,Not Applicable Imported Designed for safe ind...,Tools &amp; Home Improvement,20030513,13.90,B00009W3HD,[https://images-na.ssl-images-amazon.com/image...,3.745583,283


In [3]:
df.shape

(24035, 11)

In [4]:
threshold=37
df_new=df.query('ratingcount > @threshold')
df_new.reset_index(drop=True,inplace=True)

In [5]:
def popular_products(ratingArray, search, search_word, no_of_products, brand, category, pricehigh, pricelow, top_rated, recent):
    # df_new=reading_data()
    df_user =  pd.read_json(ratingArray)
    
    df_user = df_user.loc[:,['asin', 'clicks','rating']]
    df_user['rating']=pd.to_numeric(df_user['rating'])
    clicks_per_product = df_user.groupby('asin')['clicks'].sum()
    new_data=pd.DataFrame(clicks_per_product.values, columns= ['clicks'])
    new_data['asin']= clicks_per_product.index
    new_data['new_rating_total']=df_user.groupby('asin')['rating'].sum().values
    new_data['new_rating_num']=df_user.groupby('asin')['rating'].count().values
    
    global df_new
    # df_new = df_new.merge(new_data, left_on='asin', right_on='asin', how='left').fillna(0)
    df_merged = df_new.merge(new_data, on='asin', how='left').fillna(0)
    df_merged['rating'] = pd.to_numeric(df_merged['rating'])
    df_merged['ratingcount'] = pd.to_numeric(df_merged['ratingcount'])
    df_merged['new_rating_no'] = pd.to_numeric(df_merged['new_rating_num'])
    df_merged['new_rating_sum'] = pd.to_numeric(df_merged['new_rating_total'])
    df_merged['rating']=(df_merged['rating']*df_merged['ratingcount'] + df_merged['new_rating_sum'])/(df_merged['new_rating_no']+df_merged['ratingcount'])
    df_merged['ratingcount']=df_merged['ratingcount']+df_merged['new_rating_no']
    
    C = df_merged['ratingcount']
    R = df_merged['rating']
    avg = df_merged['rating'].mean()
    mini = df_merged['ratingcount'].quantile(0.3)
    df_merged['weighted_rating']= ((R*C)+(avg*mini))/(C+mini)
    # df_merged['no_of_clicks']=0
    df_merged[['normalized_weighted_rating','normalized_clicks']] = scaling.fit_transform(df_merged[['weighted_rating','clicks']])
    df_merged['score'] = df_merged['normalized_weighted_rating'] * 0.8 + df_merged['normalized_clicks'] * 0.2

    #Criteria for popular products
    if (df_merged['clicks'].sum()<50):
        popularity_df = df_merged.sort_values('weighted_rating', ascending = False)
    else:    
        popularity_df = df_merged.sort_values(['score'], ascending=False)
    popularity_df.reset_index(drop=True,inplace=True)
#         return popularity_df

    if(brand!=None):
        brand = [each.lower() for each in brand]
        popularity_df = popularity_df.loc[popularity_df.brand.str.lower().isin(brand)]
    if(popularity_df.shape[0]==0):
        return pd.DataFrame().to_json(orient='records')
    
    if(category!=None):
        popularity_df = popularity_df.loc[popularity_df.main_cat.str.lower()==category.lower()]
    if(popularity_df.shape[0]==0):
        return pd.DataFrame().to_json(orient='records')

    if(search):
        df1 = pd.DataFrame()
        for i in popularity_df['title']:
            if (i.lower().find(search_word.lower())!=-1):
                df1=pd.concat([df1, popularity_df.loc[popularity_df.title.str.lower()==i.lower()]], axis=0)
        popularity_df = df1
#         return df1.iloc[0: no_of_products]
    if(popularity_df.shape[0]==0):
        return pd.DataFrame().to_json(orient='records')

    if((pricehigh != None) and (pricelow != None)):
        popularity_df = popularity_df.loc[popularity_df.price < pricehigh]
        popularity_df = popularity_df.loc[popularity_df.price > pricelow]   
    elif((pricehigh == None) and (pricelow != None)):
        popularity_df = popularity_df.loc[popularity_df.price > pricelow]
    elif((pricehigh != None) and (pricelow == None)):
        popularity_df = popularity_df.loc[popularity_df.price < pricehigh]
    if(popularity_df.shape[0]==0):
        return pd.DataFrame().to_json(orient='records')
    
    if(top_rated):
        popularity_df = popularity_df.iloc[0: 1000].sort_values('rating', ascending = False)
    
    if(recent):
        return popularity_df.iloc[0: 1000].sort_values('date', ascending = False).iloc[0: no_of_products, 0:11].to_json(orient= 'index')   
    else:
        return popularity_df.iloc[0: no_of_products, 0:11].to_json(orient= 'index')
    

In [6]:
def findProduct(id):
    return df_new.loc[df_new['asin']==id].iloc[:,0:11].to_json(orient= 'records')

In [8]:
pd.read_json(findProduct("B009NQRHEY"))

,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,rating,ratingcount
0,"[, <b>Fit Compatible: </b>Samsung Galaxy S9<br...",Compatible Samsung Galaxy S9 Screen Protector ...,LuettBiden,COMPATIBLE WITH Samsung Galaxy S9:Tempered Gl...,Cell Phones & Accessories,20180813,8.98,B009NQRHEY,[https://images-na.ssl-images-amazon.com/image...,4.272727,429


In [9]:
recommendations = popular_products(ratingArray,True,'screen protector', 25,None, None, None,5, False, True )

In [10]:
rec = pd.read_json(recommendations, orient="index")
rec

,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,rating,ratingcount
3746,"[, <b>Fit Compatible: </b>Samsung Galaxy S9<br...",Compatible Samsung Galaxy S9 Screen Protector ...,LuettBiden,COMPATIBLE WITH Samsung Galaxy S9:Tempered Gl...,Cell Phones & Accessories,20180813,8.980000,B009NQRHEY,[https://images-na.ssl-images-amazon.com/image...,4.272727,429
5193,[Description the new revolution with the x-for...,POETIC Revolution X-form Design Shock Resist H...,Poetic,Made of premium polycarbonate and TPU materia...,Computers,20171021,12.950000,B01C6JC8DO,[https://images-na.ssl-images-amazon.com/image...,4.333333,102
406,[SPARIN Screen Protector with [Scratch Resist]...,"[2-PACK] Galaxy J3 (2016) Screen Protector, SP...",SPARIN,1. Perfect Fit: Precise cut dimensions provid...,All Electronics,20161119,18.059253,B01GHYP9U8,[https://images-na.ssl-images-amazon.com/image...,4.701220,164
6031,[SPARIN Zenfone 3 Deluxe Tempered Glass Screen...,SPARIN 2 PACK ZenFone 3 Deluxe ZS570KL (5.7 in...,SPARIN,Compatible For: Specially designed for honor ...,All Electronics,20161119,7.810000,B01GE38X5U,[https://images-na.ssl-images-amazon.com/image...,4.369231,65
7711,[SPARIN Galaxy S6 Active Tempered Glass Screen...,"SPARIN Galaxy S6 Active Screen Protector, (NOT...",SPARIN,Compatible For: Specially designed for Galaxy...,All Electronics,20161020,18.059253,B01GHEQ31M,[https://images-na.ssl-images-amazon.com/image...,4.320000,50
7730,[<b>Made for Fitbit Blaze</b><br> The soft sil...,Fitbit Blaze Accessory Band Large (6.1-9.3 In)...,Ztotop,Top woven stainless steel mesh loop with adju...,All Electronics,20160616,18.059253,B01DZMTX34,[https://images-na.ssl-images-amazon.com/image...,4.375000,40
6902,[Sparin High Definition Tempered Glass Screen ...,OPUDA SPARIN Huawei Honor 5X Tempered Glass Sc...,OPUDA,Perfect Matches: Made for Honor 5x with premi...,All Electronics,20160616,18.059253,B01ABO99MO,[https://images-na.ssl-images-amazon.com/image...,4.327869,61
3461,[Compatibility: Only compatible with the LG G ...,LG G Pad X8.3 Screen Protector (Verizon 4G LTE...,JOTO,Exclusive for the LG G Pad X8.3 (Verizon 4G L...,Computers,20160616,18.059253,B0131N2QPY,[https://images-na.ssl-images-amazon.com/image...,4.471264,87
6734,[<b>Package Included:</b> <br> 2 Screen Prote...,[2 Pack] Samsung Galaxy J3 2016 Screen Protect...,Luminira,0.26mm tempered glass screen protector design...,All Electronics,20160601,18.059253,B01GFU2DRG,[https://images-na.ssl-images-amazon.com/image...,4.461538,39
4771,[<b>Package Included:</b> <br> 2 Screen Prote...,[2 Pack] Samsung Galaxy Grand Prime Screen Pro...,NEARPOW,0.26mm tempered glass screen protector design...,All Electronics,20160601,18.059253,B01GFUVNH2,[https://images-na.ssl-images-amazon.com/image...,4.553191,47
